In [19]:
#coding: utf-8
from pymongo import MongoClient
from py2neo import Graph
import re
from pprint import pprint
client = MongoClient('10.243.98.93', 27017)
db = client.wikidata
coll = db.science_test

In [20]:
from py2neo import Graph
from py2neo import Node

graph = Graph(host='10.243.98.93', user='neo4j', password='admin')

nodes = {}

for n in coll.find():
    nodes[n['_id']] = Node('Concept',
                          name=n['label'])

In [21]:
from py2neo import Relationship


for l in nodes.values():
    graph.create(l)

for n in coll.find():
    if n.get('family'):
        if n['family'].get('P31'):
                for v in n['family']['P31']:
                    if v['id'] in nodes and n['_id'] in nodes:
                        graph.create(Relationship(nodes[v['id']], 'P31', nodes[n['_id']]))
        if n['family'].get('P279'):
                for v in n['family']['P279']:
                    if v['id'] in nodes and n['_id'] in nodes:
                        graph.create(Relationship(nodes[v['id']], 'P279', nodes[n['_id']]))

Pour trouver les catégories associées à une entité wikidata :

In [36]:
import pywikibot
from pprint import pprint
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

item = pywikibot.ItemPage(repo, 'Q194294')
j = item.get()
pprint(j['labels']['fr'])

u'bio\xe9thique'


In [43]:
print(len(j['claims']['P910']))
j['claims']['P910'][0].getTarget().id

1


u'Q6445103'

In [111]:
name = pywikibot.ItemPage(repo, 'Q6445103').get()['labels']['en']

Pour trouver les sous catégories : 

In [112]:
cat = pywikibot.Category(pywikibot.Site('en', 'wikipedia'), name)
cat.subcategoriesList()

[Category(Category:Animal testing),
 Category(Category:Animal welfare),
 Category(Category:Autism rights movement),
 Category(Category:Bioethicists),
 Category(Category:Bioethics journals),
 Category(Category:Bioethics research organizations),
 Category(Category:Biopunk),
 Category(Category:Biosecurity),
 Category(Category:Eugenics),
 Category(Category:Genetic engineering),
 Category(Category:Hastings Center Fellows),
 Category(Category:Health law),
 Category(Category:History of bioethics),
 Category(Category:Medical ethics),
 Category(Category:Philosophy of sexuality)]

In [131]:
cs = pywikibot.Category(pywikibot.Site('en', 'wikipedia'), 'Category:Scientific_disciplines')
count = 0
for i in cs.subcategories(recurse=3):
    count += 1
print(count)

1439


In [138]:
import pywikibot
import os
import codecs
import pickle
import re

from pywikibot.tools import open_archive
from pywikibot import config, i18n

from py2neo import Graph, Node, Relationship

class CategoryDatabase(object):

    """Temporary database saving pages and subcategories for each category.
    This prevents loading the category pages over and over again.
    """

    def __init__(self, rebuild=False, filename='category.dump.bz2'):
        """Initializer."""
        if not os.path.isabs(filename):
            filename = config.datafilepath(filename)
        self.filename = filename
        if rebuild:
            self.rebuild()

    @property
    def is_loaded(self):
        """Return whether the contents have been loaded."""
        return hasattr(self, 'catContentDB') and hasattr(self, 'superclassDB')

    def _load(self):
        if not self.is_loaded:
            try:
                if config.verbose_output:
                    pywikibot.output('Reading dump from %s'
                                     % config.shortpath(self.filename))
                with open_archive(self.filename, 'rb') as f:
                    databases = pickle.load(f)
                # keys are categories, values are 2-tuples with lists as
                # entries.
                self.catContentDB = databases['catContentDB']
                # like the above, but for supercategories
                self.superclassDB = databases['superclassDB']
                del databases
            except Exception:
                # If something goes wrong, just rebuild the database
                self.rebuild()

    def rebuild(self):
        """Rebuild the dabatase."""
        self.catContentDB = {}
        self.superclassDB = {}

    def getSubcats(self, supercat):
        """Return the list of subcategories for a given supercategory.
        Saves this list in a temporary database so that it won't be loaded from
        the server next time it's required.
        """
        self._load()
        # if we already know which subcategories exist here
        if supercat in self.catContentDB:
            return self.catContentDB[supercat][0]
        else:
            subcatset = set(supercat.subcategories())
            articleset = set(supercat.articles())
            # add to dictionary
            self.catContentDB[supercat] = (subcatset, articleset)
            return subcatset

    def getArticles(self, cat):
        """Return the list of pages for a given category.
        Saves this list in a temporary database so that it won't be loaded from
        the server next time it's required.
        """
        self._load()
        # if we already know which articles exist here
        if cat in self.catContentDB:
            return self.catContentDB[cat][1]
        else:
            subcatset = set(cat.subcategories())
            articleset = set(cat.articles())
            # add to dictionary
            self.catContentDB[cat] = (subcatset, articleset)
            return articleset

    def getSupercats(self, subcat):
        """Return the supercategory (or a set of) for a given subcategory."""
        self._load()
        # if we already know which subcategories exist here
        if subcat in self.superclassDB:
            return self.superclassDB[subcat]
        else:
            supercatset = set(subcat.categories())
            # add to dictionary
            self.superclassDB[subcat] = supercatset
            return supercatset

    def dump(self, filename=None):
        """Save the dictionaries to disk if not empty.
        Pickle the contents of the dictionaries superclassDB and catContentDB
        if at least one is not empty. If both are empty, removes the file from
        the disk.
        If the filename is None, it'll use the filename determined in __init__.
        """
        if filename is None:
            filename = self.filename
        elif not os.path.isabs(filename):
            filename = config.datafilepath(filename)
        if self.is_loaded and (self.catContentDB or self.superclassDB):
            pywikibot.output(u'Dumping to %s, please wait...'
                             % config.shortpath(filename))
            databases = {
                'catContentDB': self.catContentDB,
                'superclassDB': self.superclassDB
            }
            # store dump to disk in binary format
            with open_archive(filename, 'wb') as f:
                try:
                    pickle.dump(databases, f, protocol=config.pickle_protocol)
                except pickle.PicklingError:
                    pass
        else:
            try:
                os.remove(filename)
            except EnvironmentError:
                pass
            else:
                pywikibot.output(u'Database is empty. %s removed'
                                 % config.shortpath(filename))

    def dump_neo(self):
        graph = Graph(host='10.243.98.93', user='neo4j', password='admin')

class CategoryTreeRobot(object):

    """Robot to create tree overviews of the category structure.
    Parameters:
        * catTitle - The category which will be the tree's root.
        * catDB    - A CategoryDatabase object
        * maxDepth - The limit beyond which no subcategories will be listed.
                     This also guarantees that loops in the category structure
                     won't be a problem.
        * filename - The textfile where the tree should be saved; None to print
                     the tree to stdout.
    """

    def __init__(self, catTitle, catDB, filename=None, maxDepth=10, lang='en', source='wikipedia'):
        """Initializer."""
        self.catTitle = catTitle
        self.catDB = catDB
        if filename and not os.path.isabs(filename):
            filename = config.datafilepath(filename)
        self.filename = filename
        self.maxDepth = maxDepth
        self.site = pywikibot.Site(lang, source)

    def treeview(self, cat, currentDepth=0, parent=None):
        """Return a tree view of all subcategories of cat.
        The multi-line string contains a tree view of all subcategories of cat,
        up to level maxDepth. Recursively calls itself.
        Parameters:
            * cat - the Category of the node we're currently opening
            * currentDepth - the current level in the tree (for recursion)
            * parent - the Category of the category we're coming from
        """
        result = u'#' * currentDepth
        if currentDepth > 0:
            result += u' '
        result += re.search('Category:(.*)', cat.title()).group(1)
        result += ' (%d)' % cat.categoryinfo['pages']
        if currentDepth < self.maxDepth // 2:
            # noisy dots
            pywikibot.output('.', newline=False)
        # Create a list of other cats which are supercats of the current cat
        supercat_names = [re.search('Category:(.*)', super_cat.title()).group(1)
                          for super_cat in self.catDB.getSupercats(cat)
                          if super_cat != parent]
        if supercat_names:
            # print this list, separated with commas, using translations
            # given in 'category-also-in'
            comma = self.site.mediawiki_message('comma-separator')
            result += ' ' + comma.join(supercat_names)
        del supercat_names
        result += '\n'

        if currentDepth < self.maxDepth:
            for subcat in self.catDB.getSubcats(cat):
                # recurse into subdirectories
                result += self.treeview(subcat, currentDepth + 1, parent=cat)
        elif self.catDB.getSubcats(cat):
            # show that there are more categories beyond the depth limit
            result += '#' * (currentDepth + 1) + ' [...]\n'
        return result

    def run(self):
        """Handle the multi-line string generated by treeview.
        After string was generated by treeview it is either printed to the
        console or saved it to a file.
        """
        cat = pywikibot.Category(self.site, self.catTitle)
        pywikibot.output('Generating tree...', newline=False)
        tree = self.treeview(cat)
        pywikibot.output(u'')
        if self.filename:
            pywikibot.output(u'Saving results in %s' % self.filename)
            with codecs.open(self.filename, 'a', 'utf-8') as f:
                f.write(tree)
        else:
            pywikibot.stdout(tree)

In [139]:
catDB = CategoryDatabase(rebuild=True)
bot = CategoryTreeRobot('Computer_science', catDB, maxDepth=1)
bot.run()

Generating tree...

Computer science (44) Categories requiring diffusion, Electronic engineering, Electrical engineering, Computer engineering, Computing
# Computational fields of study (100) Systems thinking, Applied mathematics, Applied disciplines, Futurology, Systems theory, Theories of deduction, Computing and society, Knowledge representation, Big data
## [...]
# Areas of computer science (3) Subfields by academic discipline, Commons category with local link different than on Wikidata
## [...]
# Computational geometry (62) Computational fields of study, Geometry, Computational mathematics
## [...]
# Unsolved problems in computer science (24) Theoretical computer science, Unsolved problems in mathematics, Scientific problems
## [...]
# Philosophy of computer science (5) Philosophy of technology, Philosophy of mathematics
## [...]
# Computer science stubs (689) Stub categories, Science stubs, Computing stubs
## [...]
# Computer scientists (113) Mathematicians by field, Computer specialists by field, S

In [140]:
catDB.superclassDB

{Category(Category:Areas of computer science): {Category(Category:Commons category with local link different than on Wikidata),
  Category(Category:Computer science),
  Category(Category:Subfields by academic discipline)},
 Category(Category:Computational fields of study): {Category(Category:Applied disciplines),
  Category(Category:Applied mathematics),
  Category(Category:Big data),
  Category(Category:Computer science),
  Category(Category:Computing and society),
  Category(Category:Futurology),
  Category(Category:Knowledge representation),
  Category(Category:Systems theory),
  Category(Category:Systems thinking),
  Category(Category:Theories of deduction)},
 Category(Category:Computational geometry): {Category(Category:Computational fields of study),
  Category(Category:Computational mathematics),
  Category(Category:Computer science),
  Category(Category:Geometry)},
 Category(Category:Computer science): {Category(Category:Categories requiring diffusion),
  Category(Category:Comp